In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [ ]:
def alpha_m(v):
    if abs(v+54)>1.0e-8:
        alpha_m = 0.32*(v+54)/(1-np.exp(-(v+54)/4))
    else:
        alpha_m=1
    return alpha_m

In [ ]:
def beta_m(v):
    beta_m = 0.28*(v+27)/(np.exp((v+27)/5)-1)
    return beta_m

In [ ]:
def alpha_n(v):
    if abs(v+52)>1.0e-8:
        alpha_n = 0.032*(v+52)/(1-np.exp(-(v+52)/5))
    else:
        alpha_n=0.32
    return alpha_n

In [ ]:
def beta_n(v):
    beta_n = 0.5*np.exp((v+57)/40)
    return beta_n

In [ ]:
def HodgkinHuxley_2D(t,y):
    v = y[0]
    n = y[1]

    v_0=-70 # initial condition for v

    m_inf = alpha_m(v_0)/(alpha_m(v_0)+beta_m(v_0))
    n_inf = alpha_n(v_0)/(alpha_n(v_0)+beta_n(v_0))
    tau_n = 1/(alpha_n(v_0)+beta_n(v_0))

    c = 1
    g_K = 80
    g_Na = 100
    g_L = 0.1
    v_K = -100
    v_Na = 50
    v_L = -67
    I_app = 1.5

    dvdt = ( g_Na * m_inf(v)**3 * (0.83 - n) * (v_Na - v) + g_K * n**4*(v_K - v) + g_L * (v_L - v) + I_app ) / c
    dndt = (n_inf(v) - n) / tau_n

    return np.array([dvdt, dndt])